In [1]:
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [2]:
train_df = pd.read_excel("data/train.xlsx")
test1_df = pd.read_excel("data/testA.xlsx")
test2_df = pd.read_excel("data/testB.xlsx")

In [3]:
train_y = train_df[train_df.columns[-1]]
train_y.shape

(800,)

In [4]:
train_df = train_df[train_y>1.8]
train_df.shape

(799, 5954)

In [5]:
train_y = train_y[train_y>1.8]
train_y.shape

(799,)

In [6]:
new_column = test1_df.columns
train_df = train_df[new_column]
train_id = train_df['ID']
test1_id = test1_df['ID']
test2_id = test2_df['ID']
train_df.drop(['ID'], axis=1, inplace=True)
test1_df.drop(['ID'], axis=1, inplace=True)
test2_df.drop(['ID'], axis=1, inplace=True)

In [7]:
names = train_df.columns
dfs = train_df.dtypes

In [8]:
# ints or strings
ints = []
year = []
for i in range(len(names)):
    if 'int' in str(dfs[i]) or 'object' in str(dfs[i]):
        ints.append(names[i])
print(len(ints))

for n in ints:
    if str(train_df[n][0])[:4] == '2017' and str(train_df[n][499])[:4] == '2017':
        year.append(n)
print(len(year))

770
45


In [9]:
df_X = pd.concat([train_df, test1_df, test2_df], axis=0)

In [10]:
dfx_year = df_X[year]

In [11]:
remains1 = []
remains2 = []
tobedrop = []

for n in ints:
    lu1 = len(train_df[n].unique())
    lu2 = len(df_X[n].unique())
    if lu1 < 10 and lu1 > 1:
        remains1.append(n)
    if lu2 < 10 and lu2 > 1:
        remains2.append(n)
    if lu1 != lu2:
        tobedrop.append(n)
    
print(len(remains1), len(remains2), len(tobedrop), len(year))

175 281 407 45


In [12]:
toberemain = list(set(remains1) & set(remains2) - set(tobedrop))
print(len(toberemain))

84


In [13]:
def dict2list(dic:dict):
    ''' 将字典转化为列表 '''
    keys = dic.keys()
    vals = dic.values()
    lst = [(key, val) for key, val in zip(keys, vals)]
    return lst

def LastRemainJudge(column):
    unique = set(column)
    uf = {}
    for u in unique:
        uf[u] = 0
    for c in column:
        uf[c] += 1
    sl = sorted(dict2list(uf), key = lambda x:x[1], reverse=True)
    x = sl[0][1]/len(column)
    sl = np.array(sl)
    if x < 0.8:
        #print(sl[:, 1]/len(column))
        return True
    else:
        return False

In [14]:
lm1 = []
lm2 = []
for n in toberemain:
    if LastRemainJudge(train_df[n]):
        lm1.append(n)
    if LastRemainJudge(df_X[n]):
        lm2.append(n)
print(len(lm1), len(lm2), len(set(lm1)&set(lm2)))

53 52 52


In [15]:
names = df_X.columns
dts = df_X.dtypes
dfx_float_n = []
dfx_onehot_n = list(set(lm1) & set(lm2))
print(len(dfx_onehot_n))
for i in range(len(names)):
    n = names[i]
    d = dts[i]
    if 'float' in str(d):
        dfx_float_n.append(n)
print(len(dfx_float_n))
dfx_float = df_X[dfx_float_n]
dfx_onehot = df_X[dfx_onehot_n]
print(dfx_float.shape, dfx_onehot.shape)

52
5423
(1511, 5423) (1511, 52)


In [16]:
drop_names = []
names = dfx_float.columns
for n in names:
    x = np.sum(dfx_float[n].isnull())
    if x > 700:
        drop_names.append(n)
print(len(drop_names))

267


In [17]:
dfx_float.drop(drop_names, axis=1, inplace=True)

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
means = []
stds = []
drop_names = []
names = dfx_float.columns
for n in names:
    c = dfx_float[n]
    r = c[c.notnull()]
    tmpm = np.mean(r)
    tmps = np.std(r)
    means.append(tmpm)
    stds.append(tmps)
    if tmps == 0:
        drop_names.append(n)
print(len(set(means)), len(set(stds)))
print(len(drop_names))

2604 2541
364


In [19]:
dfx_float.drop(drop_names, axis=1, inplace=True)

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
# not move sames first, fill nan
names = dfx_float.columns
added = set()
sames = []
for i in range(len(names)):
    tmpsl = []
    tmpsl.append(i)
    for j in range(i+1, len(names)):
        if means[i] == means[j] and stds[i] == stds[j]:
            if i not in added:
                added.add(i)
            if j not in added:
                added.add(j)
                tmpsl.append(j)
    if len(tmpsl) > 1:
        sames.append(tmpsl)
print(len(sames))

687


In [21]:
names = dfx_float.columns
drop_names = []
for i in sames:
    for j in i[1:]:
        drop_names.append(names[j])
print(len(drop_names))

2220


In [22]:
dfx_float.drop(drop_names, axis=1, inplace=True)

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
print(dfx_float.shape)

(1511, 2572)


In [24]:
import copy
trainx_float = copy.copy(dfx_float)
#trainx_float.drop(drop_names, axis=1, inplace=True)
print(trainx_float.shape)

(1511, 2572)


In [25]:
names = trainx_float.columns
for n in names:
    c = trainx_float[n]
    r = c[c.notnull()]
    tmpm = np.mean(r)
    tmps = np.std(r)
    assert tmps>0
    trainx_float[n][c.isnull()] = tmpm
print(np.sum(trainx_float.isnull().values))

0


In [26]:
# it shall be that there is only thre lines with nan in log1p
mid = trainx_float-np.min(trainx_float)+1
trainx_log1p = np.log1p(mid)
del mid
print(np.sum(trainx_log1p.isnull().values))

0


In [27]:
for n in names:
    m1 = np.mean(trainx_float[n])
    s1 = np.std(trainx_float[n])
    assert s1 > 0
    trainx_float[n] = (trainx_float[n]-m1)/s1
    m2 = np.mean(trainx_log1p[n])
    s2 = np.std(trainx_log1p[n])
    assert s2 > 0
    trainx_log1p[n] = (trainx_log1p[n]-m2)/s2

In [28]:
ohnames = dfx_onehot.columns
for i in range(len(ohnames)):
    n = ohnames[i]
    tmp = pd.get_dummies(dfx_onehot[n], prefix='onehot_'+str(i))
    dfx_onehot = pd.concat([dfx_onehot, tmp], axis=1)

In [29]:
dfx_onehot.drop(ohnames, axis=1, inplace=True)
print(dfx_onehot.shape)

(1511, 200)


In [30]:
dfx_float = trainx_float
dfx_log1p = trainx_log1p

In [31]:
year_n = dfx_year.columns
standardlen = len(str(dfx_year[year_n[0]][0]))
for i in range(1, len(year_n)):
    l = len(str(dfx_year[year_n[i]][0]))
    bias = standardlen - l
    b10 = 1
    if bias > 0:
        for j in range(bias):
            b10 *= 10
            #dfx_year[year_n[i]] /= 10
    elif bias < 0:
        for j in range(-bias):
            b10 /= 10
            #dfx_year[year_n[i]] *= 10
    if b10 != 1:
        dfx_year[year_n[i]] *= b10
    dfx_year[year_n[i]] = dfx_year[year_n[i]] - dfx_year[year_n[0]] 

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [32]:
dfx_time = (dfx_year - np.mean(dfx_year))/np.std(dfx_year)

In [33]:
dfx_time.shape

(1511, 45)

In [34]:
dfx_time = dfx_time[dfx_time.columns[1:]]
dfx_time.shape

(1511, 44)

In [35]:
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance

/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [36]:
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

In [37]:
def MSE(y_raw, y_pred):
    r = np.mean(np.square(y_raw-y_pred))
    #print(r)
    return r

In [38]:
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV 

/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [39]:
from sklearn.model_selection import KFold 
kf = KFold(n_splits=5, random_state=2018)
def get_oof(rgr, xtrain, ytrain, xtest, lenot, lenoe):
    oof_train = np.zeros((lenot,))
    oof_test = np.zeros((lenoe,))
    oof_test_skf = np.empty((5, lenoe))
    for i, (train_index, test_index) in enumerate(kf.split(xtrain)):
        kf_xtrain = xtrain[train_index]
        kf_ytrain = ytrain[train_index]
        kf_xtest = xtrain[test_index]
        rgr.fit(kf_xtrain, kf_ytrain)
        oof_train[test_index] = rgr.predict(kf_xtest)
        oof_test_skf[i, : ] = rgr.predict(xtest)
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1,1), oof_test.reshape(-1,1)

In [40]:
dfx = pd.concat([dfx_log1p, dfx_onehot], axis=1)
dfx.shape

(1511, 2772)

In [41]:
ndfx = pd.concat([dfx, dfx_time], axis=1)
ndfx.shape

(1511, 2816)

In [66]:
trndfx = ndfx[:800]
trndfx = trndfx[train_y>1.8]

In [67]:
trny = train_y[train_y>1.8]
trny.shape

(799,)

In [42]:
# drop one point first then mnipulate the data
results = []
for i in range(100):
    subxtrain, subxval, subytrain, subyval = train_test_split(ndfx[:799].values, train_y.values, test_size=0.2, random_state=i)
    
    xgb = XGBRegressor(gamma=0.0001, max_depth=4, objective='reg:linear')
    xgb.fit(subxtrain, subytrain)
    test1_pred = xgb.predict(subxval)
    t = MSE(test1_pred, subyval)
    results.append(t)
print(np.mean(results), np.std(results), np.max(results), np.min(results))

0.0142321754574 0.00171075087503 0.0189760014693 0.010342441345


In [46]:
xgb = XGBRegressor(gamma=0.0001, max_depth=4, objective='reg:linear')
xgb.fit(ndfx[:799].values, train_y.values)
test1_pred = xgb.predict(ndfx[:799].values)
print(MSE(test1_pred, train_y.values))
test1_pred = xgb.predict(ndfx[799:1099].values)
test1_pred = test1_pred.reshape(300)
save = pd.DataFrame({'ID':test1_id, 'value':test1_pred})
save.to_csv('answer/noSynchronous_logohtime_xgb_20180120_01.csv', index=False, header=False)

0.00115347477557


In [48]:
tuned_parameters = {'objective':['reg:linear', 'reg:gamma'], 'max_depth':[1,2,3,4],
                    'gamma':[0, 1e-3, 1e-2,  1e-4], 'subsample':[0.85, 0.7]}
for i in range(100):
    subxtrain, subxval, subytrain, subyval = train_test_split(ndfx[:799].values, train_y.values, test_size=0.27, random_state=i)
    
    clf = GridSearchCV(estimator =XGBRegressor(), param_grid =tuned_parameters, cv=5, scoring='neg_mean_squared_error')
    clf.fit(subxtrain, subytrain)
    print(i)
    print(clf.best_params_)  
    y_pred1 = clf.predict(subxtrain)
    mse1 = MSE(y_pred1, subytrain)
    y_pred2 = clf.predict(subxval)
    mse2 = MSE(subyval, y_pred2)
    print(mse1, mse2)
print("----------------------------------")
clf = GridSearchCV(estimator =XGBRegressor(), param_grid =tuned_parameters, cv=5, scoring='neg_mean_squared_error')
clf.fit(trndfx.value, trny.values)
print(clf.best_params_)  
y_pred1 = clf.predict(trndfx.value)
print(MSE(y_pred1, trny.values))

0
{'gamma': 0.01, 'max_depth': 3, 'objective': 'reg:linear', 'subsample': 0.85}
0.00221182562624 0.0157196673508
1
{'gamma': 0.0001, 'max_depth': 4, 'objective': 'reg:gamma', 'subsample': 0.85}
0.000955975833544 0.0168398909048
2
{'gamma': 0.01, 'max_depth': 3, 'objective': 'reg:linear', 'subsample': 0.85}
0.00209228183326 0.0149764078866
3
{'gamma': 0.01, 'max_depth': 3, 'objective': 'reg:linear', 'subsample': 0.85}
0.00217236459476 0.0140400958242
4
{'gamma': 0.001, 'max_depth': 3, 'objective': 'reg:linear', 'subsample': 0.85}
0.00226651959531 0.0151655328155
5
{'gamma': 0.01, 'max_depth': 4, 'objective': 'reg:linear', 'subsample': 0.85}
0.000852467303127 0.0136390656625
6
{'gamma': 0, 'max_depth': 3, 'objective': 'reg:linear', 'subsample': 0.85}
0.00204433272349 0.014569950792
7
{'gamma': 0.01, 'max_depth': 3, 'objective': 'reg:linear', 'subsample': 0.7}
0.00231280041922 0.0166878607083
8
{'gamma': 0.001, 'max_depth': 3, 'objective': 'reg:linear', 'subsample': 0.85}
0.00215241054335

73
{'gamma': 0, 'max_depth': 4, 'objective': 'reg:linear', 'subsample': 0.7}
0.000712952001903 0.0150725246838
74
{'gamma': 0.0001, 'max_depth': 3, 'objective': 'reg:linear', 'subsample': 0.85}
0.00215900393328 0.0138147808028
75
{'gamma': 0.001, 'max_depth': 4, 'objective': 'reg:gamma', 'subsample': 0.85}
0.0013678959787 0.0155499547027
76
{'gamma': 0.0001, 'max_depth': 4, 'objective': 'reg:linear', 'subsample': 0.7}
0.000668843754283 0.0145149224832
77
{'gamma': 0, 'max_depth': 4, 'objective': 'reg:linear', 'subsample': 0.7}
0.000643202301725 0.0180851293004
78
{'gamma': 0.0001, 'max_depth': 4, 'objective': 'reg:linear', 'subsample': 0.7}
0.00074580905641 0.0148765602351
79
{'gamma': 0.01, 'max_depth': 4, 'objective': 'reg:linear', 'subsample': 0.85}
0.000791914487227 0.0126202627426
80
{'gamma': 0.01, 'max_depth': 4, 'objective': 'reg:linear', 'subsample': 0.85}
0.000853607915329 0.0153806866305
81
{'gamma': 0.001, 'max_depth': 4, 'objective': 'reg:linear', 'subsample': 0.85}
0.0007

NameError: name 'trndfx' is not defined

In [69]:
results = []
for i in range(100):
    subxtrain, subxval, subytrain, subyval = train_test_split(trndfx.values, trny.values, test_size=0.2, random_state=i)
    
    xgb = XGBRegressor(gamma=0.0001, max_depth=4, objective='reg:linear')
    xgb.fit(subxtrain, subytrain)
    test1_pred = xgb.predict(subxval)
    t = MSE(test1_pred, subyval)
    results.append(t)
print(np.mean(results), np.std(results), np.max(results), np.min(results))

0.01425987801 0.00164559350444 0.0184795652628 0.0103416833322


In [70]:
xgb = XGBRegressor(gamma=0.0001, max_depth=4, objective='reg:linear')
xgb.fit(trndfx.values, trny.values)
test1_pred = xgb.predict(trndfx.values)
print(MSE(test1_pred, trny.values))
test1_pred = xgb.predict(ndfx[800:1100].values)
test1_pred = test1_pred.reshape(300)
save = pd.DataFrame({'ID':test1_id, 'value':test1_pred})
save.to_csv('answer/noSynchronous_logohtime_xgb_20180119.csv', index=False, header=False)

0.00123398876007


In [71]:
# 0 : GradientBoostingRegressor
# 1 : ExtraTreesRegressor
# 2 : RandomForestRegressor
# 3 : SVM Regressor(RBF Kernel)
# 4 : KNN Regressor(Distance-weighted)
# 5 : Decision TreeRegressor
# 6 : KNN Regressor (Uniform-weighted)
# 7 : LinearRegression
# 8 : SGDRegressor
# 9 : SVM Regressor(Linear Kernel)

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, SGDRegressor

def Regressor(number):
    if number == 0:
        return GradientBoostingRegressor()
    elif number == 1:
        return ExtraTreesRegressor()
    elif number == 2:
        return RandomForestRegressor()
    elif number == 3:
        return SVR(kernel='rbf')
    elif number == 4:
        return KNeighborsRegressor(weights='distance')
    elif number == 5:
        return DecisionTreeRegressor()
    elif number == 6:
        return KNeighborsRegressor(weights='uniform')
    elif number == 7:
        return LinearRegression()
    elif number == 8:
        return SGDRegressor()
    elif number == 9:
        return SVR(kernel='linear')
    else:
        return None

In [75]:
for x in range(10):
    results = []
    for i in range(100):
        subxtrain, subxval, subytrain, subyval = train_test_split(trndfx.values, trny.values, test_size=0.2, random_state=i)
        rgr = Regressor(x)
        rgr.fit(subxtrain, subytrain)
        tepred = rgr.predict(subxval)
        t = MSE(tepred, subyval)
        results.append(t)
    print(x, np.mean(results), np.std(results), np.max(results), np.min(results))

0 0.0144297094854 0.00174658291877 0.0191151781497 0.010950372674
1 0.0193927937422 0.00285699602256 0.025930588733 0.0137611943718
2 0.0203357050017 0.00272743015394 0.0270910803524 0.0140967855898
3 0.018500045709 0.00211817216644 0.0235322698117 0.0140010209954
4 0.027539221639 0.00296901301398 0.0356893601019 0.0210478399349
5 0.0369698761661 0.0043683057054 0.0474240097001 0.0255846879016
6 0.0289217696235 0.00302206176037 0.03700701334 0.0224013488826
7 0.0402042702699 0.00751995068948 0.0755559957483 0.0277384000627


/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


8 4.93491908411e+24 2.97715046903e+24 2.38007200836e+25 1.86113360659e+24
9 0.020113022857 0.00220094379958 0.0269094454547 0.0132261763663
